In [ ]:
def load_map(input_data):
    lines = input_data.strip().split('\n')
    # Find where the map ends (first empty line)
    map_end = 0
    for i, line in enumerate(lines):
        if not line.strip():
            map_end = i
            break

    map_lines = [list(line) for line in lines[:map_end]]
    moves = ''.join(lines[map_end+1:]).strip()
    return map_lines, moves

In [ ]:
def find_robot(map_grid):
    for y in range(len(map_grid)):
        for x in range(len(map_grid[y])):
            if map_grid[y][x] == '@':
                return (x, y)
    return None

In [ ]:
def find_boxes(map_grid, shape='O'):
    boxes = []
    for y in range(len(map_grid)):
        for x in range(len(map_grid[y])):
            if map_grid[y][x] in (shape):
                boxes.append((x, y))
    return boxes

In [ ]:
def find_wrong_boxes(map_grid, wrong_shape=".]"):
    wrong_boxes = []
    for y in range(len(map_grid)):
        for x in range(len(map_grid[y]) - 1):
            if map_grid[y][x] == wrong_shape[0] and map_grid[y][x+1] == wrong_shape[1]:
                wrong_boxes.append((x, y))
    return wrong_boxes

In [ ]:
def get_box_score(boxes):
    return sum(100*j + i for i, j in boxes)

In [ ]:
def get_move_delta(move):
    if move == '^': return (0, -1)
    if move == 'v': return (0, 1)
    if move == '<': return (-1, 0)
    if move == '>': return (1, 0)
    return (0, 0)

In [ ]:
def check_push_line(map_grid, start_x, start_y, dx, dy):
    """Check line of boxes in push direction, return list of box positions"""
    boxes = []
    x, y = start_x, start_y

    while map_grid[y][x] == 'O':
        boxes.append((x, y))
        x += dx
        y += dy
        if map_grid[y][x] == '#':
            return None  # Hit wall, push not possible

    if map_grid[y][x] == '.':
        return boxes
    return None

In [ ]:
def simulate_moves(map_grid, moves):
    robot_pos = find_robot(map_grid)
    map_grid[robot_pos[1]][robot_pos[0]] = '.'

    for move in moves:
        dx, dy = get_move_delta(move)
        new_x = robot_pos[0] + dx
        new_y = robot_pos[1] + dy

        # Check if move is valid
        if map_grid[new_y][new_x] == '#':
            continue

        # Check if there's a box
        if map_grid[new_y][new_x] == 'O':
            boxes = check_push_line(map_grid, new_x, new_y, dx, dy)
            if boxes:
                # Move all boxes
                for box_x, box_y in reversed(boxes):
                    map_grid[box_y + dy][box_x + dx] = 'O'
                    map_grid[box_y][box_x] = '.'
                robot_pos = (new_x, new_y)
            continue

        # Move robot
        if map_grid[new_y][new_x] == '.':
            robot_pos = (new_x, new_y)


    # Place robot in final position
    map_grid[robot_pos[1]][robot_pos[0]] = '@'
    return map_grid

In [ ]:
def print_map(map_grid):
    for row in map_grid:
        print(''.join(row))

In [ ]:
def solve(input_data):
    map_grid, moves = load_map(input_data)
    final_state = simulate_moves(map_grid, moves)
    return final_state

In [ ]:
# file = "example-mini"
# file = "example-mini-2"
# file = "example"
file = "input"

In [ ]:
with open(file) as f:
    input_data = f.read()

In [ ]:
result = solve(input_data)

In [ ]:
print_map(result)

In [ ]:
get_box_score(find_boxes(result))

In [ ]:
def double_width_map(map_grid):
    """
    Transform map grid to double width format
    Input: 2D list of characters
    Output: 2D list of characters with doubled width
    """
    new_map = []
    for row in map_grid:
        new_row = []
        for char in row:
            if char == '@':
                new_row.extend(['@', '.'])  # Robot with padding
            elif char == 'O':
                new_row.extend(['[', ']'])
            elif char == '#':
                new_row.extend(['#', '#'])
            elif char == '.':
                new_row.extend(['.', '.'])
        new_map.append(new_row)
    return new_map

In [ ]:
def check_box_at(map_grid, x, y):
    """
    Check if position is part of a box (either [ or ])
    Returns tuple ((left_x, y), (right_x, y)) if box found, None otherwise
    """
    if x < 0 or y < 0 or y >= len(map_grid) or x >= len(map_grid[0]):
        return None

    # Check if we're at left half
    if map_grid[y][x] == '[' and x+1 < len(map_grid[0]) and map_grid[y][x+1] == ']':
        return ((x, y), (x+1, y))

    # Check if we're at right half
    if x > 0 and map_grid[y][x] == ']' and map_grid[y][x-1] == '[':
        return ((x-1, y), (x, y))

    return None

In [ ]:
def find_pushable_boxes(map_grid, start_x, start_y, dx, dy):
    """Find all boxes that can be pushed from this position"""
    boxes = []  # List of ((left_x, y), (right_x, y)) tuples
    checked = set()  # Set of (x, y) tuples for all box halves

    def check_row(x, y):
        # Skip if we've checked this position
        if (x, y) in checked:
            return True

        # Get box at current position
        box = check_box_at(map_grid, x, y)
        if not box:
            return True

        (left, right) = box
        # Skip if we've checked either half of this box
        if left in checked or right in checked:
            return True

        checked.add(left)
        checked.add(right)
        boxes.append(box)

        # Calculate next positions
        next_y = y + dy

        # Check if next position hits wall
        if map_grid[next_y][left[0]] == '#' or map_grid[next_y][right[0]] == '#':
            return False

        if dy != 0:  # Vertical movement
            # Check boxes in next row that overlap with current box
            for next_x in (left[0], right[0]):
                next_box = check_box_at(map_grid, next_x, next_y)
                if next_box:
                    if not check_row(next_box[0][0], next_y):
                        return False
            return True

        else:  # Horizontal movement
            if dx < 0:
                next_x = left[0] + dx
            if dx > 0:
                next_x = right[0] + dx

            # Check if next position hits wall
            if map_grid[next_y][next_x] == '#':
                return False

            next_box = check_box_at(map_grid, next_x, next_y)
            if next_box:
                return check_row(next_box[0][0], next_y)
            return True

    # Start checking from initial position
    initial_box = check_box_at(map_grid, start_x, start_y)
    if initial_box and check_row(initial_box[0][0], start_y):
        return boxes
    return None

In [ ]:
import copy

In [ ]:
def simulate_moves(map_grid, moves):
    robot_pos = find_robot(map_grid)
    map_grid[robot_pos[1]][robot_pos[0]] = '.'

    for i, move in enumerate(moves):
        dx, dy = get_move_delta(move)
        new_x = robot_pos[0] + dx  # Remove //2 to keep original movement
        new_y = robot_pos[1] + dy

        # # Debug print
        # map_grid_plot = copy.deepcopy(map_grid)
        # map_grid_plot[robot_pos[1]][robot_pos[0]] = '@'
        # print_map(map_grid_plot)
        # print()
        # print(move)

        # Check boundaries and walls
        if map_grid[new_y][new_x] == '#':
            continue

        # Move robot
        if map_grid[new_y][new_x] == '.':
            robot_pos = (new_x, new_y)

        # Check for boxes
        if check_box_at(map_grid, new_x, new_y):
            boxes = find_pushable_boxes(map_grid, new_x, new_y, dx, dy)
            if boxes:
                # Sort boxes based on movement direction
                if dx < 0:  # Moving left, place from left to right
                    boxes.sort(key=lambda box: box[0][0])
                elif dx > 0:  # Moving right, place from right to left
                    boxes.sort(key=lambda box: box[0][0], reverse=True)
                elif dy < 0:  # Moving up, place from top to bottom
                    boxes.sort(key=lambda box: box[0][1])
                elif dy > 0:  # Moving down, place from bottom to top
                    boxes.sort(key=lambda box: box[0][1], reverse=True)

                # First clear all boxes
                for (left, right) in boxes:
                    map_grid[left[1]][left[0]] = '.'
                    map_grid[right[1]][right[0]] = '.'

                # Then place all boxes in sorted order
                for (left, right) in boxes:
                    map_grid[left[1] + dy][left[0] + dx] = '['
                    map_grid[right[1] + dy][right[0] + dx] = ']'

                robot_pos = (new_x, new_y)
            continue


    # Place robot in final position
    map_grid[robot_pos[1]][robot_pos[0]] = '@'
    return map_grid

In [ ]:
map_grid, moves = load_map(input_data)
map_grid = double_width_map(map_grid)

In [ ]:
result = simulate_moves(map_grid, moves)

In [ ]:
find_wrong_boxes(result)

In [ ]:
print_map(result)

In [ ]:
get_box_score(find_boxes(result, shape='['))

Debug cases

In [ ]:
map_grid = """
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##...[]...[]..[]..##
##[]##....[]......##
##[][]@.......[]..##
##.....[]..[].[][]##
##........[]......##
####################
""".strip().split('\n')

map_grid = [list(row) for row in map_grid]
move = "<"

In [ ]:
map_grid = """
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##...[]...[]..[]..##
##..##....[]......##
##...[].......[]..##
##....[]...[].[][]##
##.....@..[]......##
####################
""".strip().split('\n')

map_grid = [list(row) for row in map_grid]
move = "^"

In [ ]:
map_grid = """
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##...[]...[]..[]..##
##..#.[]..[]......##
##...[].......[]..##
##....[]...[].[][]##
##.....@..[]......##
####################
""".strip().split('\n')

map_grid = [list(row) for row in map_grid]
move = "^"

In [ ]:
map_grid = """
####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##...[]...[]..[]..##
##[]##....[]......##
##[][].@......[]..##
##.....[]..[].[][]##
##........[]......##
####################
""".strip().split('\n')

map_grid = [list(row) for row in map_grid]
move = "<"

In [ ]:
map_grid = """
####################
##[]..[]....[]..[]##
##[]..........[]..##
##.@[][]....[]..[]##
##...[]...[]..[]..##
##..##....[]......##
##...[].......[]..##
##.....[]..[].[][]##
##........[]......##
####################
""".strip().split('\n')

map_grid = [list(row) for row in map_grid]
move = ">"

In [ ]:
map_grid = """
####################
##[]..[]....[]..[]##
##[]..........[]..##
##.........@[][][]##
##....[]..[]..[]..##
##..##....[]......##
##...[]...[]..[]..##
##.....[]..[].[][]##
##........[]......##
####################
""".strip().split('\n')

map_grid = [list(row) for row in map_grid]
move = "v"

In [ ]:
map_grid = """
####################
##[]..[]....[]..[]##
##[]..........[]..##
##..........[][][]##
##.....[].[]..[]..##
##..##[][][]......##
##..[].[].[]..[]..##
##...[]@...[].[][]##
##........[]......##
####################
""".strip().split('\n')

map_grid = [list(row) for row in map_grid]
move = "^"

In [ ]:
result = simulate_moves(map_grid, move)
print_map(result)